In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

pd.options.display.max_columns = None
sns.set_theme(style="whitegrid")

# -----------------------------
# 0) Config
# -----------------------------
# If running locally in this repo, use this path:
DATA_DIR = '.'
# If running in Colab and you uploaded files, set DATA_DIR = '.' and ensure files exist

OUTPUT_DIR = './outputs'
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

# -----------------------------
# 1) Load Data
# -----------------------------
def load_csvs(data_dir: str):
    files = {
        'members': f'{data_dir}/members.csv',
        'coaches': f'{data_dir}/coaches.csv',
        'subscriptions': f'{data_dir}/subscriptions.csv',
        'sessions': f'{data_dir}/sessions.csv',
        'plans': f'{data_dir}/plans.csv',
        'packages': f'{data_dir}/packages.csv',
    }
    dfs = {k: pd.read_csv(v) for k, v in files.items()}
    return dfs

dfs = load_csvs(DATA_DIR)
members = dfs['members']
coaches = dfs['coaches']
subscriptions = dfs['subscriptions']
sessions = dfs['sessions']
plans = dfs['plans']
packages = dfs['packages']

# -----------------------------
# 2) Preprocess
# -----------------------------
def to_dt(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce')
    return df

members = to_dt(members, ['created_at', 'dob'])
subscriptions = to_dt(subscriptions, ['start_date', 'end_date', 'created_at', 'updated_at'])
sessions = to_dt(sessions, ['date', 'start_time', 'end_time', 'created_at', 'updated_at'])

# derive duration
subscriptions['subscription_days'] = (subscriptions['end_date'] - subscriptions['start_date']).dt.days

# normalize plan link
subs_plans = subscriptions.merge(plans, left_on='plan_id', right_on='id', how='left', suffixes=('_sub','_plan'))

# capacity if missing -> assume NA
if 'capacity' not in coaches.columns:
    coaches['capacity'] = np.nan

# -----------------------------
# 4) Member Activity Trends
# -----------------------------
print('=== 4) Member Activity Trends ===')
if not sessions.empty and 'date' in sessions.columns:
    sessions['day'] = sessions['date'].dt.date
    sessions['week'] = sessions['date'].dt.to_period('W').astype(str)
    sessions['month'] = sessions['date'].dt.to_period('M').astype(str)
    sessions['dow'] = sessions['date'].dt.day_name()
    
    # Extract hour for peak time analysis
    if 'start_time' in sessions.columns:
        sessions['hour'] = sessions['start_time'].dt.hour
        peak_hours = sessions.groupby('hour').size().rename('bookings').sort_values(ascending=False)
        print('Peak booking hours:')
        print(peak_hours.head(10))
        
        # Peak hours chart
        fig = px.bar(peak_hours.reset_index(), x='hour', y='bookings', 
                    title='Bookings by Hour of Day', 
                    labels={'hour': 'Hour of Day', 'bookings': 'Number of Bookings'})
        fig.show()
        fig.write_html(f'{OUTPUT_DIR}/bookings_by_hour.html')
    
    daily = sessions.groupby('day').size().rename('bookings')
    weekly = sessions.groupby('week').size().rename('bookings')
    monthly = sessions.groupby('month').size().rename('bookings')
    by_dow = sessions.groupby('dow').size().rename('bookings').sort_values(ascending=False)

    print('Daily bookings (head):')
    print(daily.head())
    print('Weekly bookings (head):')
    print(weekly.head())
    print('Monthly bookings:')
    print(monthly)

    # quick charts
    fig = px.line(daily.reset_index(), x='day', y='bookings', title='Daily Bookings')
    fig.show()
    fig.write_html(f'{OUTPUT_DIR}/member_daily_bookings.html')

    fig = px.bar(by_dow.reset_index(), x='dow', y='bookings', title='Bookings by Day of Week')
    fig.show()
    fig.write_html(f'{OUTPUT_DIR}/bookings_by_dow.html')
    
    # Monthly trend chart
    fig = px.bar(monthly.reset_index(), x='month', y='bookings', title='Monthly Bookings Trend')
    fig.update_xaxes(tickangle=45)
    fig.show()
    fig.write_html(f'{OUTPUT_DIR}/monthly_bookings.html')
else:
    print('No sessions date data available.')

# -----------------------------
# 5) Coach Performance Analysis
# -----------------------------
print('\n=== 5) Coach Performance Analysis ===')
# sessions by coach (model_id or instructor_id)
coach_key = 'model_id' if 'model_id' in sessions.columns else 'instructor_id' if 'instructor_id' in sessions.columns else None

if coach_key:
    coach_perf = sessions.groupby(coach_key).agg(
        total_sessions=('id', 'count'),
        cancelled=('status', lambda s: (s.astype(str).str.lower() == 'cancelled').sum())
    ).reset_index()

    # attach coach names/pricing/capacity if available
    coach_perf = coach_perf.merge(coaches.rename(columns={'id':'coach_id'}), left_on=coach_key, right_on='coach_id', how='left')
    coach_perf['cancel_rate_pct'] = np.where(coach_perf['total_sessions']>0, coach_perf['cancelled']/coach_perf['total_sessions']*100, 0)

    print(coach_perf[['coach_id','name','total_sessions','cancelled','cancel_rate_pct','price','capacity']].sort_values('total_sessions', ascending=False).head(10))

    # Look for ratings columns in sessions, coaches, or separate ratings data
    rating_columns = [col for col in sessions.columns if 'rating' in col.lower() or 'score' in col.lower() or 'review' in col.lower()]
    if rating_columns:
        print(f'\nFound rating columns: {rating_columns}')
        for rating_col in rating_columns:
            if sessions[rating_col].notna().sum() > 0:  # Check if there's actual rating data
                rating_stats = sessions.groupby(coach_key)[rating_col].agg(['mean', 'count', 'std']).reset_index()
                rating_stats = rating_stats.merge(coaches.rename(columns={'id':'coach_id'}), left_on=coach_key, right_on='coach_id', how='left')
                rating_stats.columns = [coach_key, 'avg_rating', 'rating_count', 'rating_std', 'coach_id', 'name', 'price', 'capacity']
                
                print(f'\nCoach ratings ({rating_col}):')
                print(rating_stats[['name', 'avg_rating', 'rating_count', 'rating_std']].sort_values('avg_rating', ascending=False).head(10))
                
                # Rating chart
                if 'name' in rating_stats.columns:
                    rating_stats['name'] = rating_stats['name'].fillna(f'Coach {rating_stats["coach_id"]}')
                    fig = px.bar(rating_stats, x='name', y='avg_rating', 
                               title=f'Average Coach Ratings ({rating_col})',
                               labels={'avg_rating': 'Average Rating'})
                    fig.update_xaxes(tickangle=45)
                    fig.show()
                    fig.write_html(f'{OUTPUT_DIR}/coach_ratings.html')
                break
    else:
        print('\nNo rating columns found in sessions data.')
        # Check if there might be ratings in coaches table
        coach_rating_cols = [col for col in coaches.columns if 'rating' in col.lower() or 'score' in col.lower()]
        if coach_rating_cols:
            print(f'Found rating columns in coaches table: {coach_rating_cols}')
            for col in coach_rating_cols:
                if coaches[col].notna().sum() > 0:
                    print(f'Coach ratings from coaches table ({col}):')
                    coach_ratings = coaches[['name', col]].sort_values(col, ascending=False).head(10)
                    print(coach_ratings)
                    
                    fig = px.bar(coach_ratings, x='name', y=col, 
                               title=f'Coach Ratings ({col})')
                    fig.update_xaxes(tickangle=45)
                    fig.show()
                    fig.write_html(f'{OUTPUT_DIR}/coach_ratings_from_coaches.html')
        else:
            print('No rating data found in coaches table either.')

    # Handle cases where 'name' column might be missing or have NaN values
    if 'name' in coach_perf.columns:
        coach_perf['name'] = coach_perf['name'].fillna(f'Coach {coach_perf["coach_id"]}')
        
        fig = px.bar(coach_perf, x='name', y='total_sessions', title='Sessions per Coach')
        fig.update_xaxes(tickangle=45)
        fig.show()
        fig.write_html(f'{OUTPUT_DIR}/sessions_per_coach.html')

        fig = px.bar(coach_perf, x='name', y='cancel_rate_pct', title='Cancellation Rate per Coach (%)')
        fig.update_xaxes(tickangle=45)
        fig.show()
        fig.write_html(f'{OUTPUT_DIR}/cancel_rate_per_coach.html')
    else:
        print('Coach name column not available for charting.')
else:
    print('No coach identifier column found in sessions (expected model_id or instructor_id).')

# -----------------------------
# 6) Plan & Subscription Performance
# -----------------------------
print('\n=== 6) Plan & Subscription Performance ===')
# Popular plans
if 'name' in subs_plans.columns:
    plan_counts = subs_plans['name'].value_counts(dropna=False)
    print('Popular plans:')
    print(plan_counts.head(10))
else:
    print('No plan name column found for popularity analysis.')
    plan_counts = pd.Series(dtype=int)

# Revenue by plan
if 'price_sub' in subs_plans.columns:
    price_col = 'price_sub'
elif 'price' in subs_plans.columns:
    price_col = 'price'
else:
    price_col = None

if price_col and 'name' in subs_plans.columns:
    rev_by_plan = subs_plans.groupby('name')[price_col].sum().sort_values(ascending=False)
    print('\nRevenue by plan:')
    print(rev_by_plan)
else:
    print('\nNo price column found for revenue calculation.')
    rev_by_plan = pd.Series(dtype=float)

# Subscription duration analysis
print('\nSubscription duration stats (days):')
duration_stats = subscriptions['subscription_days'].describe()
print(duration_stats)

# Enhanced churn analysis
print('\n--- Enhanced Churn Analysis ---')
if 'status' in subscriptions.columns:
    # Status-based churn
    status_churn_count = (subscriptions['status'].astype(str).str.lower() == 'cancelled').sum()
    status_churn_rate = status_churn_count / len(subscriptions) * 100 if len(subscriptions) else 0
    print(f'Status-based churn rate: {status_churn_rate:.2f}% (count={status_churn_count})')

# Time-based churn (subscriptions that have ended)
current_date = pd.Timestamp.now()
expired_subs = subscriptions[subscriptions['end_date'] < current_date]
time_churn_rate = len(expired_subs) / len(subscriptions) * 100 if len(subscriptions) else 0
print(f'Time-based churn rate (expired subscriptions): {time_churn_rate:.2f}% (count={len(expired_subs)})')

# Churn by plan type
if 'name' in subs_plans.columns and 'status' in subs_plans.columns:
    churn_by_plan = subs_plans.groupby('name').agg(
        total_subs=('id_sub', 'count'),
        cancelled_subs=('status', lambda x: (x.astype(str).str.lower() == 'cancelled').sum())
    ).reset_index()
    churn_by_plan['churn_rate'] = churn_by_plan['cancelled_subs'] / churn_by_plan['total_subs'] * 100
    print('\nChurn rate by plan:')
    print(churn_by_plan.sort_values('churn_rate', ascending=False))
    
    fig = px.bar(churn_by_plan, x='name', y='churn_rate', 
                title='Churn Rate by Plan (%)',
                labels={'churn_rate': 'Churn Rate (%)'})
    fig.update_xaxes(tickangle=45)
    fig.show()
    fig.write_html(f'{OUTPUT_DIR}/churn_by_plan.html')

# Charts - FIXED: Proper column renaming for plan_counts
if not plan_counts.empty:
    plan_counts_df = plan_counts.reset_index()
    plan_counts_df.columns = ['plan', 'count']  # Explicitly set column names

    fig = px.bar(plan_counts_df, x='plan', y='count', title='Popular Plans')
    fig.update_xaxes(tickangle=45)  # Rotate x-axis labels for better readability
    fig.show()
    fig.write_html(f'{OUTPUT_DIR}/popular_plans.html')
else:
    print('No plan data available for charting.')

# FIXED: Proper column handling for revenue by plan
if not rev_by_plan.empty:
    rev_by_plan_df = rev_by_plan.reset_index()
    rev_by_plan_df.columns = ['plan', 'revenue']  # Explicitly set column names

    fig = px.bar(rev_by_plan_df, x='plan', y='revenue', title='Revenue by Plan')
    fig.update_xaxes(tickangle=45)  # Rotate x-axis labels for better readability
    fig.show()
    fig.write_html(f'{OUTPUT_DIR}/revenue_by_plan.html')

# Subscription duration distribution
if not subscriptions['subscription_days'].isna().all():
    fig = px.histogram(subscriptions, x='subscription_days', 
                      title='Distribution of Subscription Duration (Days)',
                      labels={'subscription_days': 'Subscription Duration (Days)'})
    fig.show()
    fig.write_html(f'{OUTPUT_DIR}/subscription_duration_distribution.html')

# -----------------------------
# 7) Utilization Metrics
# -----------------------------
print('\n=== 7) Utilization Metrics ===')
# Coach availability vs bookings (needs capacity). If capacity missing, prints NA.
util = None
if coach_key:
    # Aggregate bookings per coach (scheduled only)
    scheduled = sessions[sessions['status'].astype(str).str.lower() == 'scheduled'] if 'status' in sessions.columns else sessions.copy()
    booked_per_coach = scheduled.groupby(coach_key).size().rename('booked').reset_index()
    util = booked_per_coach.merge(coaches.rename(columns={'id':'coach_id'}), left_on=coach_key, right_on='coach_id', how='left')
    util['utilization_pct'] = np.where(util['capacity'].notna() & (util['capacity']>0),
                                       util['booked']/util['capacity']*100, np.nan)
    
    print('Coach Utilization Analysis:')
    print(util[['coach_id','name','booked','capacity','utilization_pct']].sort_values('booked', ascending=False).head(10))

    # Overall utilization stats
    avg_utilization = util['utilization_pct'].mean()
    max_utilization = util['utilization_pct'].max()
    min_utilization = util['utilization_pct'].min()
    print(f'\nOverall Utilization Stats:')
    print(f'Average: {avg_utilization:.2f}%')
    print(f'Maximum: {max_utilization:.2f}%')
    print(f'Minimum: {min_utilization:.2f}%')

    # FIXED: Handle potential NaN values in utilization chart
    if 'name' in util.columns:
        util['name'] = util['name'].fillna(f'Coach {util["coach_id"]}')
        util_chart_data = util.dropna(subset=['utilization_pct'])
        
        if not util_chart_data.empty:
            fig = px.bar(util_chart_data, x='name', y='utilization_pct', 
                        title='Coach Utilization (%)', 
                        labels={'utilization_pct':'% Utilization'})
            fig.update_xaxes(tickangle=45)
            fig.show()
            fig.write_html(f'{OUTPUT_DIR}/coach_utilization.html')
        else:
            print('No valid utilization data available for charting.')
    else:
        print('Coach name column not available for utilization charting.')
else:
    print('Coach utilization requires coach_id in sessions and capacity in coaches.')

# Room/studio occupancy analysis
if 'model' in sessions.columns:
    studio_sessions = sessions[sessions['model'].astype(str).str.lower() == 'studio']
    if not studio_sessions.empty:
        by_day = studio_sessions.groupby(studio_sessions['date'].dt.date).size()
        print('\nStudio occupancy (bookings per day) head:')
        print(by_day.head())
        
        # Studio occupancy stats
        avg_daily_occupancy = by_day.mean()
        max_daily_occupancy = by_day.max()
        print(f'Average daily studio bookings: {avg_daily_occupancy:.2f}')
        print(f'Maximum daily studio bookings: {max_daily_occupancy}')
        
        # FIXED: Proper column naming for studio bookings chart
        studio_bookings_df = by_day.reset_index()
        studio_bookings_df.columns = ['date', 'bookings']
        
        fig = px.line(studio_bookings_df, x='date', y='bookings', title='Studio Bookings per Day')
        fig.show()
        fig.write_html(f'{OUTPUT_DIR}/studio_daily_bookings.html')
        
        # Studio occupancy by day of week
        studio_sessions['dow'] = studio_sessions['date'].dt.day_name()
        studio_by_dow = studio_sessions.groupby('dow').size().rename('bookings')
        fig = px.bar(studio_by_dow.reset_index(), x='dow', y='bookings', 
                    title='Studio Bookings by Day of Week')
        fig.show()
        fig.write_html(f'{OUTPUT_DIR}/studio_bookings_by_dow.html')
    else:
        print('No studio sessions found.')
else:
    print('Model column not found in sessions for studio occupancy analysis.')

# Overall facility utilization summary
print('\n--- Overall Facility Utilization Summary ---')
total_sessions = len(sessions)
total_scheduled = len(sessions[sessions['status'].astype(str).str.lower() == 'scheduled']) if 'status' in sessions.columns else total_sessions
total_cancelled = len(sessions[sessions['status'].astype(str).str.lower() == 'cancelled']) if 'status' in sessions.columns else 0
cancellation_rate = (total_cancelled / total_sessions * 100) if total_sessions > 0 else 0

print(f'Total sessions: {total_sessions}')
print(f'Scheduled sessions: {total_scheduled}')
print(f'Cancelled sessions: {total_cancelled}')
print(f'Overall cancellation rate: {cancellation_rate:.2f}%')

